Задание 1 Реализация нейронной сети с двумя сверточными слоями и одним полносвязным с кусочко-линейной функцией активации.

1) Подключение библиотек

In [11]:
import os
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import TensorDataset, DataLoader

2) Настройка вычислений

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("device:", device)

torch.manual_seed(42)

device: cuda


3) Загрузка подготовленного датасета

In [13]:
DATA_PATH = r"..\data\prepared\notmnist_28.npz"
assert os.path.exists(DATA_PATH), f"File not found: {DATA_PATH}"

d = np.load(DATA_PATH)

X_train = torch.tensor(d["X_train"], dtype=torch.float32)
y_train = torch.tensor(d["y_train"], dtype=torch.long)

X_val = torch.tensor(d["X_val"], dtype=torch.float32)
y_val = torch.tensor(d["y_val"], dtype=torch.long)

X_test = torch.tensor(d["X_test"], dtype=torch.float32)
y_test = torch.tensor(d["y_test"], dtype=torch.long)

print("train:", X_train.shape, y_train.shape)
print("val:  ", X_val.shape, y_val.shape)
print("test: ", X_test.shape, y_test.shape)

train: torch.Size([415752, 1, 28, 28]) torch.Size([415752])
val:   torch.Size([46194, 1, 28, 28]) torch.Size([46194])
test:  torch.Size([13649, 1, 28, 28]) torch.Size([13649])


4) Фомирование DataLoader

In [14]:
BATCH_SIZE = 128

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader   = DataLoader(TensorDataset(X_val, y_val),     batch_size=BATCH_SIZE, shuffle=False, num_workers=0)
test_loader  = DataLoader(TensorDataset(X_test, y_test),   batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

5) Определение архитектуры нейронной сети (2 сверточных слоя + 1 полносвязный слой, активация кусочко-линейная)

In [15]:
class SimpleConvNet(nn.Module):
    
    def __init__(self, activation="relu"):
        super().__init__()

        self.conv1 = nn.Conv2d(1, 8, kernel_size=5, padding=2)
        self.conv2 = nn.Conv2d(8, 16, kernel_size=5, padding=2)
        
        self.fc = nn.Linear(16 * 28 * 28, 10)

        if activation == "relu":
            self.act = nn.ReLU()
        elif activation == "leakyrelu":
            self.act = nn.LeakyReLU(0.1)
        else:
            raise ValueError("activation must be 'relu' or 'leakyrelu'")

    def forward(self, x):
        x = self.act(self.conv1(x))
        x = self.act(self.conv2(x))
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

6) Проверка корректности размерностей

In [16]:
model = SimpleConvNet(activation="relu").to(device)

xb, yb = next(iter(train_loader))
xb = xb.to(device)

out = model(xb)
print("out shape:", out.shape)

out shape: torch.Size([128, 10])


7) Оптимизатор и гиперпараметры обучения

In [17]:
LR = 1e-3
EPOCHS = 30

optimizer = torch.optim.Adam(model.parameters(), lr=LR)

8) Функции обучения и оценки качества

In [18]:
def accuracy(logits, y):
    preds = logits.argmax(dim=1)
    return (preds == y).float().mean().item()


@torch.no_grad()
def evaluate(model, loader):
    model.eval()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)
        logits = model(X)
        loss = F.cross_entropy(logits, y)

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n


def train_one_epoch(model, loader, optimizer):
    model.train()
    total_loss, total_acc, n = 0.0, 0.0, 0

    for X, y in loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()
        logits = model(X)
        loss = F.cross_entropy(logits, y)
        loss.backward()
        optimizer.step()

        bs = X.size(0)
        total_loss += loss.item() * bs
        total_acc += accuracy(logits, y) * bs
        n += bs

    return total_loss / n, total_acc / n

9) Обучение модели

In [19]:
for epoch in range(1, EPOCHS + 1):
    train_loss, train_acc = train_one_epoch(model, train_loader, optimizer)
    val_loss, val_acc = evaluate(model, val_loader)

    print(f"Epoch {epoch:02d}: "
          f"train loss={train_loss:.4f}, acc={train_acc:.4f} | "
          f"val loss={val_loss:.4f}, acc={val_acc:.4f}")

Epoch 01: train loss=0.4374, acc=0.8779 | val loss=0.3809, acc=0.8942
Epoch 02: train loss=0.3568, acc=0.9002 | val loss=0.3587, acc=0.9018
Epoch 03: train loss=0.3274, acc=0.9076 | val loss=0.3491, acc=0.9032
Epoch 04: train loss=0.3070, acc=0.9129 | val loss=0.3480, acc=0.9041
Epoch 05: train loss=0.2903, acc=0.9175 | val loss=0.3542, acc=0.9022
Epoch 06: train loss=0.2772, acc=0.9209 | val loss=0.3577, acc=0.9026
Epoch 07: train loss=0.2653, acc=0.9240 | val loss=0.3624, acc=0.9026
Epoch 08: train loss=0.2553, acc=0.9268 | val loss=0.3663, acc=0.9035
Epoch 09: train loss=0.2468, acc=0.9287 | val loss=0.3694, acc=0.9022
Epoch 10: train loss=0.2385, acc=0.9307 | val loss=0.3780, acc=0.9011
Epoch 11: train loss=0.2314, acc=0.9326 | val loss=0.3905, acc=0.9008
Epoch 12: train loss=0.2255, acc=0.9340 | val loss=0.3987, acc=0.9001
Epoch 13: train loss=0.2194, acc=0.9359 | val loss=0.4106, acc=0.8997
Epoch 14: train loss=0.2141, acc=0.9372 | val loss=0.4140, acc=0.8979
Epoch 15: train loss

10) Оценка качества на тестовой выборке

In [20]:
test_loss, test_acc = evaluate(model, test_loader)
print(f"TEST: loss={test_loss:.4f}, acc={test_acc:.4f}")

TEST: loss=0.2596, acc=0.9369


В ходе обучения модели наблюдается рост точности на обучающей выборке от 0.878 до 0.949, что свидетельствует об успешной оптимизации. Однако уже после 4-5 эпох начинает проявляться переобучение, значение функции потерь на валидации увеличивается с 0.348 до 0.540, а точность на валидационной выборке снижается с 0.904 до 0.890. Следовательно, оптимальное число эпох для данной архитектуры составляет около 3-6, а дальнейшее обучение не улучшает качество обобщения. Точность на тестовой выборке после 30 эпох составила 0.937, что ниже возможного максимума из-за эффекта переобучения.